In [126]:
from polygon import RESTClient
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import datetime
import numpy as np
load_dotenv()

portfolio_data = pd.DataFrame()


## Importing Polyon Stock Data

In [127]:
def load_portfolio_data(ticker : str, name: str, type : str, portfolio_data: pd.DataFrame,):
    client = RESTClient(os.environ['polygon_api_key'])
    aggs = client.get_aggs(
        ticker,
        1,
        "day",
        "2023-01-01",
        str(datetime.date.today()),
        limit=5000
    )
    df = pd.DataFrame(aggs).assign(ticker=ticker, name=name, type=type)
    return pd.concat([portfolio_data, df])

In [128]:
portfolio_data = load_portfolio_data(
    ticker="TSM",
    name="Taiwan Semiconductor Mfg. Co. Ltd.",
    type="stock",
    portfolio_data=portfolio_data
)

In [129]:
portfolio_data = load_portfolio_data(
    ticker="AMZN",
    name="Amazon.com Inc",
    type="stock",
    portfolio_data=portfolio_data
)

In [130]:
portfolio_data = load_portfolio_data(
    ticker="NVDA",
    name="NVIDIA Corp",
    type="stock",
    portfolio_data=portfolio_data
)

In [131]:
portfolio_data = load_portfolio_data(
    ticker="AXP",
    name="American Express Company",
    type="stock",
    portfolio_data=portfolio_data
)

In [132]:
portfolio_data = load_portfolio_data(
    ticker="CELH",
    name="Celsius Holdings, Inc.",
    type="stock",
    portfolio_data=portfolio_data
)

In [133]:
portfolio_data = load_portfolio_data(
    ticker="C:USDJPY",
    name="USD/JPY Foreign Exchange",
    type="forex",
    portfolio_data=portfolio_data
)

In [134]:
portfolio_data = load_portfolio_data(
    ticker="SPY",
    name="SPDR S&P 500 ETF Trust",
    type="etf",
    portfolio_data=portfolio_data
)

In [230]:
client = RESTClient(os.environ['polygon_api_key'])
NDX_data = pd.DataFrame(client.get_aggs(
    'I:NDX',
    1,
    "day",
    "2023-01-01",
    str(datetime.date.today()),
    limit=5000
))

In [135]:
# Check to make sure all stock data is loaded in
portfolio_data["ticker"].unique()

array(['TSM', 'AMZN', 'NVDA', 'AXP', 'CELH', 'C:USDJPY', 'SPY'],
      dtype=object)

## Data Cleaning

In [136]:
# resetting the index
portfolio_data.reset_index(drop=True, inplace=True)

In [137]:
# dropping the otc column
portfolio_data.drop(columns=["otc"], inplace=True)

In [138]:
# converting epoch times to dates
portfolio_data["timestamp"] = pd.to_datetime(portfolio_data["timestamp"], unit="ms").dt.date
portfolio_data = portfolio_data.rename(columns={'timestamp': 'date'})
portfolio_data["date"] = pd.to_datetime(portfolio_data["date"])


In [139]:
# formatting numbers
portfolio_data = portfolio_data.round(2)
portfolio_data["volume"] = portfolio_data["volume"].astype(int)

In [239]:
# cleaning index data
NDX_data.drop(columns=["volume","vwap","transactions","otc"], inplace=True)
NDX_data = NDX_data.rename(columns={'timestamp' : 'date'})
NDX_data['date'] = pd.to_datetime(NDX_data['date'], unit='ms').dt.date
NDX_data['date'] = pd.to_datetime(NDX_data['date'])
NDX_data = NDX_data.round(2)

NDX_data

,open,high,low,close,date
0,12085.67,12156.21,12006.04,12066.27,2023-02-22
1,11979.80,12018.32,11900.84,11969.65,2023-02-24
2,12106.79,12159.64,12034.61,12057.79,2023-02-27
3,12041.75,12146.52,12021.32,12042.12,2023-02-28
4,12026.72,12054.48,11906.58,11937.48,2023-03-01
...,...,...,...,...,...
343,19995.28,20186.63,19995.28,20186.63,2024-07-03
344,20224.13,20406.99,20201.50,20391.97,2024-07-05
345,20393.89,20455.38,20363.37,20439.54,2024-07-08
346,20504.17,20543.90,20395.57,20453.02,2024-07-09


## Outlier Cleaning

In [140]:
# Checking for outliers in transactions and volume
portfolio_data.loc[portfolio_data["ticker"] == "C:USDJPY"].describe()

,open,high,low,close,volume,vwap,date,transactions
count,499.000000,499.000000,499.000000,499.000000,499.000000,499.000000,499,499.000000
mean,144.369739,144.882766,143.845511,144.437735,152266.703407,144.411463,2023-09-30 19:51:49.418837504,152266.703407
min,127.350000,127.880000,127.350000,127.870000,1.000000,127.880000,2023-01-01 00:00:00,1.000000
25%,136.595000,137.505000,136.285000,136.970000,109867.000000,136.805000,2023-05-13 12:00:00,109867.000000
50%,145.710000,146.220000,144.940000,145.730000,180092.000000,145.570000,2023-09-27 00:00:00,180092.000000
75%,150.095000,150.560000,149.800000,150.135000,213513.000000,150.105000,2024-02-17 00:00:00,213513.000000
max,161.580000,162.000000,161.300000,161.590000,357136.000000,161.590000,2024-07-11 00:00:00,357136.000000
std,8.498044,8.437833,8.542451,8.499978,86947.436292,8.487131,NaN,86947.436292


In [141]:
# Removing all weekend days
portfolio_data.drop(index=portfolio_data[portfolio_data["date"].dt.day_of_week>4].index, inplace=True)

# Removing all days the stock market is not open
no_holidays = portfolio_data['date'].value_counts() == 7
portfolio_data = portfolio_data[portfolio_data['date'].isin(no_holidays[no_holidays].index)]


## Aggregations

Calculating how many shares of what to buy (buying based off vwap and partial shares are allowed)
- 40% SPY
- 10% USDJPY
- 10% AMZN
- 10% TSM
- 10% CELH
- 10% AXP
- 10% NVDA

In [142]:
SPY_shares = 40000/portfolio_data[portfolio_data["ticker"] == "SPY"]["vwap"].iat[0]
USDJPY_shares = 10000/portfolio_data[portfolio_data["ticker"]=="C:USDJPY"]["vwap"].iat[0]
AMZN_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AMZN"]["vwap"].iat[0]
TSM_shares = 10000/portfolio_data[portfolio_data["ticker"]=="TSM"]["vwap"].iat[0]
CELH_shares = 10000/portfolio_data[portfolio_data["ticker"]=="CELH"]["vwap"].iat[0]
AXP_shares = 10000/portfolio_data[portfolio_data["ticker"]=="AXP"]["vwap"].iat[0]
NVDA_shares = 10000/portfolio_data[portfolio_data["ticker"]=="NVDA"]["vwap"].iat[0]

Creating dataframes for each investment

In [143]:
cols = ['date','open','high','low','close','volume','vwap','transactions']
SPY_etf_data = portfolio_data[portfolio_data["ticker"] == "SPY"][cols].reset_index(drop=True)
USDJPY_forex_data = portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][cols].reset_index(drop=True)
AMZN_stock_data = portfolio_data[portfolio_data["ticker"] == "AMZN"][cols].reset_index(drop=True)
TSM_stock_data = portfolio_data[portfolio_data["ticker"] == "TSM"][cols].reset_index(drop=True)
AXP_stock_data = portfolio_data[portfolio_data["ticker"] == "AXP"][cols].reset_index(drop=True)
CELH_stock_data = portfolio_data[portfolio_data["ticker"] == "CELH"][cols].reset_index(drop=True)
NVDA_stock_data = portfolio_data[portfolio_data["ticker"] == "NVDA"][cols].reset_index(drop=True)

In [144]:

portfolio = pd.concat([
    portfolio_data[portfolio_data["ticker"] == "SPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "C:USDJPY"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AMZN"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "TSM"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "AXP"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "CELH"][['ticker','name','type']].head(1),
    portfolio_data[portfolio_data["ticker"] == "NVDA"][['ticker','name','type']].head(1)
]).reset_index(drop=True)

portfolio

,ticker,name,type
0,SPY,SPDR S&P 500 ETF Trust,etf
1,C:USDJPY,USD/JPY Foreign Exchange,forex
2,AMZN,Amazon.com Inc,stock
3,TSM,Taiwan Semiconductor Mfg. Co. Ltd.,stock
4,AXP,American Express Company,stock
5,CELH,"Celsius Holdings, Inc.",stock
6,NVDA,NVIDIA Corp,stock


Performing Stock/ETF/Forex aggregations

In [225]:
SPY_etf_data['value'] = (SPY_etf_data['vwap'] * SPY_shares).round(2)
SPY_etf_data['cumulative return'] = (SPY_etf_data['value'] - SPY_etf_data['value'].iat[0])/SPY_etf_data['value'].iat[0]
SPY_etf_data['volatility'] = SPY_etf_data['vwap'].rolling(len(SPY_etf_data),min_periods=2).std()

AMZN_stock_data['value'] = (AMZN_stock_data['vwap'] * AMZN_shares).round(2)
AMZN_stock_data['cumulative return'] = (AMZN_stock_data['value'] - AMZN_stock_data['value'].iat[0])/AMZN_stock_data['value'].iat[0]
AMZN_stock_data['volatility'] = AMZN_stock_data['vwap'].rolling(len(AMZN_stock_data),min_periods=2).std()

TSM_stock_data['value'] = (TSM_stock_data['vwap'] * TSM_shares).round(2)
TSM_stock_data['cumulative return'] = (TSM_stock_data['value'] - TSM_stock_data['value'].iat[0])/TSM_stock_data['value'].iat[0]
TSM_stock_data['volatility'] = TSM_stock_data['vwap'].rolling(len(TSM_stock_data),min_periods=2).std()

AXP_stock_data['value'] = (AXP_stock_data['vwap'] * AXP_shares).round(2)
AXP_stock_data['cumulative return'] = (AXP_stock_data['value'] - AXP_stock_data['value'].iat[0])/AXP_stock_data['value'].iat[0]
AXP_stock_data['volatility'] = AXP_stock_data['vwap'].rolling(len(AXP_stock_data),min_periods=2).std()

CELH_stock_data['value'] = (CELH_stock_data['vwap'] * CELH_shares).round(2)
CELH_stock_data['cumulative return'] = (CELH_stock_data['value'] - CELH_stock_data['value'].iat[0])/CELH_stock_data['value'].iat[0]
CELH_stock_data['volatility'] = CELH_stock_data['vwap'].rolling(len(CELH_stock_data),min_periods=2).std()

NVDA_stock_data['value'] = (NVDA_stock_data['vwap'] * NVDA_shares).round(2)
NVDA_stock_data['cumulative return'] = (NVDA_stock_data['value'] - NVDA_stock_data['value'].iat[0])/NVDA_stock_data['value'].iat[0]
NVDA_stock_data['volatility'] = NVDA_stock_data['vwap'].rolling(len(NVDA_stock_data),min_periods=2).std()


USDJPY_forex_data['value'] = (USDJPY_forex_data['vwap'] * USDJPY_shares).round(2)
USDJPY_forex_data['cumulative return'] = (USDJPY_forex_data['value'] - USDJPY_forex_data['value'].iat[0])/USDJPY_forex_data['value'].iat[0]
USDJPY_forex_data['percent change'] = USDJPY_forex_data['value'].pct_change()
USDJPY_forex_data.loc[0, 'percent change'] = 0

Performing portfolio aggregations

In [227]:
portfolio_aggs = pd.DataFrame(SPY_etf_data['date'])
portfolio_aggs['value'] = (
    SPY_etf_data['value'] +
    AMZN_stock_data['value'] +
    TSM_stock_data['value'] +
    AXP_stock_data['value'] +
    CELH_stock_data['value'] +
    NVDA_stock_data['value'] +
    USDJPY_forex_data['value']
)
portfolio_aggs['cumulative return'] = (portfolio_aggs['value'] - portfolio_aggs['value'].iat[0])/portfolio_aggs['value'].iat[0]

portfolio_aggs['annualized return'] = (
    (1 + portfolio_aggs['cumulative return']).pow(365/(portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days) - 1
)

less_than_1_year = (portfolio_aggs['date'] - portfolio_aggs['date'].iat[0]).dt.days < 365
# Global Investment Performance Standards dictate that returns of portfolios or composites for periods of less than one year may not be annualized
portfolio_aggs.loc[less_than_1_year, 'annualized return'] = 0

portfolio_aggs['volatility'] = portfolio_aggs['value'].rolling(len(portfolio_aggs),min_periods=2).std()
portfolio_aggs

,date,value,cumulative return,annualized return,volatility
0,2023-01-03,100000.00,0.000000,0.000000,NaN
1,2023-01-04,100821.29,0.008213,0.000000,580.739728
2,2023-01-05,100104.03,0.001040,0.000000,447.176589
3,2023-01-06,101043.70,0.010437,0.000000,518.134235
4,2023-01-09,103385.28,0.033853,0.000000,1369.403536
...,...,...,...,...,...
376,2024-07-03,237825.46,1.378255,0.782659,38122.953031
377,2024-07-05,240030.02,1.400300,0.789854,38317.841564
378,2024-07-08,241367.70,1.413677,0.790769,38517.195700
379,2024-07-09,242538.41,1.425384,0.794606,38719.994541
